# Importing and Installing libraries

In [ ]:
!git clone https://github.com/leekunhee/Mask_RCNN.git
!cd Mask_RCNN && python setup.py install

In [ ]:
import os,sys
import pandas as pd
from PIL import Image
from os import listdir
from numpy import zeros, asarray, expand_dims, mean
import numpy as np

ROOT_DIR = os.path.abspath("./Mask_RCNN")
sys.path.append(ROOT_DIR) 

from mrcnn.utils import Dataset,extract_bboxes
from mrcnn.visualize import display_instances
from matplotlib import pyplot
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

import warnings
warnings.filterwarnings("ignore")

# Dataset

In [ ]:
class PedDataset(Dataset):
    def load_dataset(self, dataset_dir='../input/pennfudan-database-for-pedestrian-detection-zip/PennFudanPed', mode='train'):
        self.add_class('dataset',1,'pedestrian')
        images_dir = dataset_dir + '/PNGImages/'
        annotations_dir = dataset_dir + '/Annotation/'
        masks_dir = dataset_dir + '/PedMasks/'
        if mode=='train':
            j=0
            for filename in listdir(images_dir):
                j+=1
                if j>150:
                    continue
                image_id = filename
                img_path = images_dir + image_id
                ann_path = annotations_dir + image_id[:-4]+'.txt'
                mask_path = masks_dir + image_id[:-4]+'_mask.png'
                self.add_image('dataset', image_id=image_id, path=img_path, ann_path=ann_path, mask_path=mask_path)
        if mode=='test':
            j=0
            for filename in listdir(images_dir):
                j+=1
                if j<=150:
                    continue
                image_id = filename
                img_path = images_dir + image_id
                ann_path = annotations_dir + image_id[:-4]+'.txt'
                mask_path = masks_dir + image_id[:-4]+'_mask.png'
                self.add_image('dataset', image_id=image_id, path=img_path, ann_path=ann_path, mask_path=mask_path)
        
    def extract_boxes(self, image_id):
        info = self.image_info[image_id]
        file = info['ann_path']
        boxes = list()
        f = open(filename,'r')
        for x in f:
            if '(Xmin, Ymin) - (Xmax, Ymax)' in x:
                xmin = int(x[-23:-20])
                ymin = int(x[-18:-15])
                xmax = int(x[-10:-7])
                ymax = int(x[-5:-2])
                coors = [xmin, ymin, xmax, ymax]
                boxes.append(coors)
        width = 559
        height = 536
        return boxes, width, height
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        file = info['mask_path']
        #boxes, w, h = self.extract_boxes(image_id)
        #masks = zeros([w, h, len(boxes)], dtype='uint8')
        masks = Image.open(file)
        masks = np.array(masks)
        ob_id = np.unique(masks)
        ob_id = ob_id[1:]
        masks = masks == ob_id[:, None, None]
        masks = np.moveaxis(masks,0,-1)
        class_ids = list()
        for i in range(len(ob_id)):
#             box = boxes[i]
#             row_s, row_e = box[1], box[3]
#             col_s, col_e = box[0], box[2]
#             masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('pedestrian'))
        return masks, asarray(class_ids, dtype='int32')
    
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
train_set = PedDataset()
train_set.load_dataset(mode='train')
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
 
test_set = PedDataset()
test_set.load_dataset(mode='test')
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [ ]:
image_id = 2
image = train_set.load_image(image_id)
print(image.shape)
mask, class_ids = train_set.load_mask(image_id)
print(mask.shape)
pyplot.imshow(image)
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.6)
pyplot.show()

# Training

In [ ]:
class PedConfig(Config):
    NAME = "ped_cfg"
    NUM_CLASSES = 2
    STEPS_PER_EPOCH = 150
    VALIDATION_STEPS = 20
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 384
    IMAGE_MAX_DIM = 448
    
config = PedConfig()

In [ ]:
config.display()

In [ ]:
model = MaskRCNN(mode='training', model_dir='./', config=config)

In [ ]:
model.load_weights('../input/mask-rcnn-coco-weights/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=10, layers='heads')

# Evaluation

In [ ]:
class PredictionConfig(Config):
    NAME = "ped_cfg"
    NUM_CLASSES = 2
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    USE_MINI_MASK = False

In [ ]:
cfg = PredictionConfig()
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)

In [ ]:
for i in listdir():
    print(i)
    if i[:3]=='ped':
        path=i

In [ ]:
model.load_weights('./'+path+'/mask_rcnn_ped_cfg_0010.h5', by_name=True)

In [ ]:
def evaluate_model(dataset, model, cfg):
    APs = list()
    j=0
    for image_id in dataset.image_ids:
        if j>50:
            continue
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id)
        scaled_image = mold_image(image, cfg)
        sample = expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose=0)
        r = yhat[0]
        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
        APs.append(AP)
        j+=1
    mAP = mean(APs)
    return mAP

In [ ]:
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

In [ ]:
class_names = ['BG', 'pedestrian']
for i in range(5):
    image = test_set.load_image(i)
    results = model.detect([image], verbose=1)
    r = results[0]
    display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])